In [1]:
import os
import h5py
import numpy as np
import cv2 as cv
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, ReLU, MaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

Generate customized masks from trained U-Net (+ VGG16)

In [2]:
# Download pretrained weights of VGG16 from the web
weight_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                      'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [3]:
IMG_SIZE = 128

def create_model():
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    conv1_1 = Conv2D(64, 3, 1, 'SAME', activation='relu')(inputs)
    conv1_2 = Conv2D(64, 3, 1, 'SAME', activation='relu')(conv1_1)
    pool1_3 = MaxPooling2D()(conv1_2)
    
    conv2_1 = Conv2D(128, 3, 1, 'SAME', activation='relu')(pool1_3)
    conv2_2 = Conv2D(128, 3, 1, 'SAME', activation='relu')(conv2_1)
    pool2_3 = MaxPooling2D()(conv2_2)
    
    conv3_1 = Conv2D(256, 3, 1, 'SAME', activation='relu')(pool2_3)
    conv3_2 = Conv2D(256, 3, 1, 'SAME', activation='relu')(conv3_1)
    conv3_3 = Conv2D(256, 3, 1, 'SAME', activation='relu')(conv3_2)
    pool3_4 = MaxPooling2D()(conv3_3)
    
    conv4_1 = Conv2D(512, 3, 1, 'SAME', activation='relu')(pool3_4)
    conv4_2 = Conv2D(512, 3, 1, 'SAME', activation='relu')(conv4_1)
    conv4_3 = Conv2D(512, 3, 1, 'SAME', activation='relu')(conv4_2)
    pool4_4 = MaxPooling2D()(conv4_3)
    
    conv5_1 = Conv2D(512, 3, 1, 'SAME', activation='relu')(pool4_4)
    conv5_2 = Conv2D(512, 3, 1, 'SAME', activation='relu')(conv5_1)
    conv5_3 = Conv2D(512, 3, 1, 'SAME', activation='relu')(conv5_2)
    pool5_4 = MaxPooling2D()(conv5_3)
    
    # Load the weights of pretrained VGG16
    vgg = keras.Model(inputs, pool5_4)
    vgg.load_weights(weight_path)
    
    upconv6 = Conv2DTranspose(512, 5, 2, 'SAME', activation='relu')(pool5_4)
    concat6 = Concatenate()([conv5_3, upconv6])
    conv6 = Conv2D(512, 3, 1, 'SAME', activation='relu')(concat6)
                              
    upconv7 = Conv2DTranspose(512, 5, 2, 'SAME', activation='relu')(conv6)
    concat7 = Concatenate()([conv4_3, upconv7])
    conv7 = Conv2D(512, 3, 1, 'SAME', activation='relu')(concat7)
    
    upconv8 = Conv2DTranspose(256, 5, 2, 'SAME', activation='relu')(conv7)
    concat8 = Concatenate()([conv3_3, upconv8])
    conv8 = Conv2D(256, 3, 1, 'SAME', activation='relu')(concat8)
    
    upconv9 = Conv2DTranspose(128, 5, 2, 'SAME', activation='relu')(conv8)
    concat9 = Concatenate()([conv2_2, upconv9])
    conv9 = Conv2D(128, 3, 1, 'SAME', activation='relu')(concat9)
    
    upconv10 = Conv2DTranspose(64, 5, 2, 'SAME', activation='relu')(conv9)
    concat10 = Concatenate()([conv1_2, upconv10])
    conv10 = Conv2D(64, 3, 1, 'SAME', activation='relu')(concat10)
    
    conv11 = Conv2D(64, 3, 1, 'SAME', activation='relu')(conv10)
    
    conv12 = Conv2D(2, 1, 1, 'SAME', activation='softmax')(conv11)
    
    return keras.Model(inputs=inputs, outputs=conv12) 

model = create_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 64  36928       ['conv2d[0][0]']                 
                                )                                                           

In [ ]:
train_test = 'train'
checkpoint_path = '[USER_PATH]\\ArtificialVision\\checkpoints'
weight_path = os.path.join(checkpoint_path, 'saved-unet_model-02-0.09.hdf5')
model.load_weights(weight_path)

data_path = '[USER_PATH]\\ArtificialVision\\data'
input_path = os.path.join(data_path, 'Middle_Resolution_177_unzipped_parcropped_128')
imagePaths = [os.path.join(input_path, file_name) for file_name in os.listdir(input_path)]
output_path = os.path.join(data_path, f'Middle_Resolution_177_unzipped_parcropped_128_mask_{train_test}')
os.makedirs(output_path, exist_ok=True)

In [ ]:
if train_test == 'train':
    camera_list = list(map(int, [4,5,6,7,8,9,10,14,15,16,17,18,19,20]))
    light_list = list(map(int, [1,2,3,4,5,6, 
                                8,9,10,11,12,13,14,15, 16,17,18,
                                19,20,21,22,23,24,25,26,27, 28,29,30])) 
    accessory_list = list(map(int, [1,2,3,5])) 
    expression_list = list(map(int, list(range(1, 4))))

elif train_test == 'test':
    camera_list = ['4','7','10'] 
    light_list = ['1']
    accessory_list =['1']
    expression_list = ['1','2','3'] 

for imagePath in imagePaths: # for all facial classes
    print(imagePath)
    
    folder_name = os.path.basename(imagePath) # facial class ID
    os.makedirs(output_path+'\\'+folder_name, exist_ok=True) 
    person_output_path = output_path + '\\' + folder_name 
    filePaths = [os.path.join(imagePath, file_name) for file_name in os.listdir(imagePath)] # for all files in one facial class

    if os.listdir(person_output_path):
      print(f'Already done with {person_output_path}')
      pass
    else:
      print(imagePath)
      for filePath in filePaths: 
        file_name  = os.path.basename(filePath)
            
        light_split = file_name.split('L')
        exp_split = light_split[-1].split('E')
        ang_split = file_name.split('C')
        dot_split = ang_split[-1].split('.')
        
        if (int(light_split[0][-1]) in accessory_list) and (int(exp_split[0]) in light_list) and (int(ang_split[0][-1]) in expression_list) and (int(dot_split[0]) in camera_list):
          image = Image.open(file_name)
          image = image.resize((224, 224))
          image = np.array(image)
          image = image/255. 
          image = np.reshape(image, (1, 224, 224, 3))

          prediction = model.predict(image)
          pred = np.zeros_like(prediction)
          thr = 0.5
          pred[prediction>=thr] = 255 # facial features
          pred[prediction<thr] = 0 # background

          mask = pred[0,:,:,1]
          mask = cv.resize(mask, dsize=(128, 128), interpolation=cv.INTER_CUBIC) 

          head_and_tail = os.path.splitext(os.path.basename(file_name)) 
          cv.imwrite(person_output_path+'\\'+head_and_tail[0], mask) 

Remove backgrounds using the created masks

In [ ]:
train_test = 'train'
data_path = '[USER_PATH]\\ArtificialVision\\data'
image_path = os.path.join(data_path, 'Middle_Resolution_177_unzipped_parcropped_128')
mask_path = os.path.join(data_path, f'Middle_Resolution_177_unzipped_parcropped_128_mask_{train_test}')
output_path = os.path.join(data_path, f'Middle_Resolution_177_unzipped_parcropped_128_removed_{train_test}')
os.makedirs(output_path, exist_ok=True)

In [ ]:
if train_test == 'train':
    camera_list = list(map(int, [4,5,6,7,8,9,10,14,15,16,17,18,19,20]))
    light_list = list(map(int, [1,2,3,4,5,6, 
                                8,9,10,11,12,13,14,15, 16,17,18,
                                19,20,21,22,23,24,25,26,27, 28,29,30])) 
    accessory_list = list(map(int, [1,2,3,5])) 
    expression_list = list(map(int, list(range(1, 4))))

elif train_test == 'test':
    camera_list = ['4','7','10'] 
    light_list = ['1']
    accessory_list =['1']
    expression_list = ['1','2','3'] 

imagePaths = [os.path.join(image_path,file_name) for file_name in os.listdir(image_path)]
maskPaths = [os.path.join(mask_path,file_name) for file_name in os.listdir(mask_path)] 

IMG_SIZE = 128

for imagePath, maskPath in zip(imagePaths, maskPaths):

    folder_name = os.path.basename(imagePath)
    os.makedirs(output_path+'\\'+folder_name, exist_ok=True)
    person_output_path = output_path + '\\' + folder_name 
    image_filePaths = [os.path.join(imagePath, file_name) for file_name in os.listdir(imagePath)] 
    mask_filePaths = [os.path.join(maskPath, file_name) for file_name in os.listdir(maskPath)] 
   
    if os.listdir(person_output_path):
        print(f'Already done with {person_output_path}')
        pass
    else:
        print(imagePath)
        for image_filePath in image_filePaths:

            file_name = os.path.basename(image_filePath)
                    
            light_split = file_name.split('L')
            exp_split = light_split[-1].split('E')
            ang_split = file_name.split('C')
            dot_split = ang_split[-1].split('.')

            if (int(light_split[0][-1]) in accessory_list) and (int(exp_split[0]) in light_list) and (int(ang_split[0][-1]) in expression_list) and (int(dot_split[0]) in camera_list):
                img = cv.imread(image_filePath) 
                mask_filePath = os.path.join(maskPath, file_name)
                mask = cv.imread(mask_filePath)
                mask = mask
               
                idx_location = np.zeros((IMG_SIZE*IMG_SIZE, 2)); 
                k=0
                x, y = img.shape[0], img.shape[1] 

                for i in range(0,x):
                    for j in range(0,y):
                        if mask[i,j,0]==0 and mask[i,j,1]==0 and mask[i,j,2]==0: 
                            idx_location[k,0]=i 
                            idx_location[k,1]=j 
                            k = k+1
                idx_location = idx_location[0:k,:]
                idx_n = k # the number of zero pixels per image 

                for k in range(0,idx_n):
                    ii = int(idx_location[k,0])
                    jj = int(idx_location[k,1])
                    img[ii,jj,0] = 0
                    img[ii,jj,1] = 0
                    img[ii,jj,2] = 0 

                cv.imwrite(person_output_path+'\\'+os.path.basename(file_name), img)